Derive Orbital Elements
<br>
<br>
    - Semi-major axis: The distance from the center of an ellipse to its furthest edge, along the major axis
<br>    
    - Eccentricity: Measure of how elliptical the orbit is around the sun, and is represented by a value between 0 and 1: 
<br>
    Eccentricity of 0: A perfect circle 
<br>
    Eccentricity of 1: A parabola
<br>
<br>
    - Inclination: The angle between the plane of the orbit and the ecliptic plane(Plane of Earth's orbit around the sun)
<br>
<br>
    - Longitude of the ascending node: The angle between the reference direction and the point at which the orbit crosses the ecliptic plane from the south to the north
<br>
<br>
    - Argument of Perihelion: The angle between the ascending node and the point at which the planet is closest to the sun (An argument of periapsis of 0° means that the orbiting body's Periapsis is at the Ascending Node. An argument of periapsis of 90° means that the orbiting body will reach periapsis at its northmost distance from the plane of reference.)

In [27]:
import pandas as pd
import numpy as np
df = pd.read_csv('RawData/horizons_results_30_years.csv')

GM = np.float128(1.32712440018e11) # km^3/s^2

df.head()

,Date,Position_x,Position_y,Position_z,Velocity_x,Velocity_y,Velocity_z
0,1964-09-01,9.802199e+07,4.642303e+07,-5.009535e+06,-16.841095,31.169176,1.515031
1,1964-09-02,9.653499e+07,4.907756e+07,-4.878073e+06,-17.559232,30.280814,1.525763
2,1964-09-03,9.499122e+07,5.165641e+07,-4.746249e+06,-18.157275,29.421917,1.523812
3,1964-09-04,9.340043e+07,5.416325e+07,-4.615044e+06,-18.650868,28.617836,1.511939
4,1964-09-05,9.177075e+07,5.660374e+07,-4.485177e+06,-19.061267,27.889125,1.493449


In [28]:
position = df[['Position_x', 'Position_y', 'Position_z']].values.astype(np.float128)
velocity = df[['Velocity_x', 'Velocity_y', 'Velocity_z']].values.astype(np.float128)

angular_momentum = np.cross(position, velocity)
vel_cross_h = np.cross(velocity, angular_momentum)

angular_momentum_magnitude = np.sqrt(angular_momentum[:,0]**2 + angular_momentum[:,1]**2 + angular_momentum[:,2]**2)
pos_magnitude = np.sqrt(np.sum(position**2, axis=1, dtype=np.float128))

In [29]:
#Caclulate semi-major axis
pos_x = df['Position_x'].values
pos_y = df['Position_y'].values
pos_z = df['Position_z'].values
vel_x = df['Velocity_x'].values
vel_y = df['Velocity_y'].values
vel_z = df['Velocity_z'].values

r_magnitude = np.sqrt(pos_x**2 + pos_y**2 + pos_z**2)
v_magnitude = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)

semi_major_axis_arr = 1/((2/r_magnitude) - (v_magnitude**2/GM))

semi_major_axis = np.mean(semi_major_axis_arr)

print(f'Mean Semi-major axis: {semi_major_axis} kms')
print("True value of semi-major axis of Venus: 108200000 km")
print("Relative error: ", ((semi_major_axis - 108200000)/108200000)*100, "%")

with open('Results/constants.txt', 'w') as f:
    f.write(f'Mean Semi-major axis: {semi_major_axis} kms\n')

Mean Semi-major axis: 109444031.56288806 kms
True value of semi-major axis of Venus: 108200000 km
Relative error:  1.1497519065508903439 %


In [30]:
periheleion = np.min(r_magnitude)
apoheleion = np.max(r_magnitude)

#convert to AU
periheleion = periheleion/149597870.7
apoheleion = apoheleion/149597870.7

print('Periheleion: ', periheleion, 'AU')
print('Apoheleion: ', apoheleion, 'AU')
print("True value of periheleion of Venus: 0.7184 AU")
print("True value of apoheleion of Venus: 0.7282 AU")
eccentricity = (apoheleion - periheleion)/(apoheleion + periheleion)
print('Eccentricity: ', eccentricity)
print("True value of eccentricity of Venus: 0.0067")
print("Relative error: ", ((eccentricity - 0.0067)/0.0067)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write('Periheleion: ' + str(periheleion) + ' AU\n')
    f.write('Apoheleion: ' + str(apoheleion) + ' AU\n')
    f.write('Eccentricity: ' + str(eccentricity) + '\n')

Periheleion:  0.7137602900595876 AU
Apoheleion:  0.7328811833264984 AU
True value of periheleion of Venus: 0.7184 AU
True value of apoheleion of Venus: 0.7282 AU
Eccentricity:  0.013217437505199834
True value of eccentricity of Venus: 0.0067
Relative error:  97.27518664477363 %


In [31]:
#calculate average inclination
inclination = np.arccos(angular_momentum[:,2]/angular_momentum_magnitude)
inclination = np.degrees(inclination)
inclination = np.mean(inclination)
print(f'Mean Inclination: {inclination} degrees')
print("True value of inclination of Venus: 3.39 degrees")
print("Relative error: ", ((inclination - 3.39)/3.39)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Inclination: {inclination} degrees\n')

Mean Inclination: 3.39415026472638 degrees
True value of inclination of Venus: 3.39 degrees
Relative error:  0.12242668809380086163 %


In [32]:
#Longitude of ascending node
omega = np.arctan2(angular_momentum[:,0], -1*angular_momentum[:,1])
omega = np.degrees(omega)
omega = np.mean(omega)
print(f'Mean Longitude of ascending node: {omega} degrees')
print("True value of Longitude of ascending node of Venus: 76.785 degrees")
print("Relative error: ", ((omega - 76.785)/76.785)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Longitude of ascending node: {omega} degrees\n')

Mean Longitude of ascending node: 76.73822747264366 degrees
True value of Longitude of ascending node of Venus: 76.785 degrees
Relative error:  -0.06091362552105618696 %


In [33]:
#Calculate Argument of periapsis
node_vector = np.cross([0,0,1], angular_momentum) #direction of the ascending node; Lies in the reference plane; Points from the central body toward the ascending node.
node_vector_magnitude = np.sqrt(node_vector[:,0]**2 + node_vector[:,1]**2 + node_vector[:,2]**2)
node_vector_unit = node_vector/node_vector_magnitude[:,np.newaxis]

e_vector = vel_cross_h/GM - position/pos_magnitude[:,np.newaxis] #eccentricity vector
e_vector_magnitude = np.sqrt(e_vector[:,0]**2 + e_vector[:,1]**2 + e_vector[:,2]**2)
e_vector_unit = e_vector/e_vector_magnitude[:,np.newaxis]

w = np.arccos(np.dot(node_vector_unit, e_vector_unit.T))
w = np.degrees(w)
w = np.mean(w)
print(f'Mean Argument of periapsis: {w} degrees')
print("True value of Argument of periapsis of Venus: 54.78 degrees")
print("Relative error: ", ((w - 54.78)/54.78)*100, "%")

with open('Results/constants.txt', 'a') as f:
    f.write(f'Mean Argument of periapsis: {w} degrees\n')

Mean Argument of periapsis: 88.27113636265724 degrees
True value of Argument of periapsis of Venus: 54.78 degrees
Relative error:  61.137525306055572966 %
